In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os
username = "user"
password = "grad5100user"
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
port = 3306

In [2]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [3]:
tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series.
The database contains information on both. We are not interested in the ‘Masters’. There
is also a season called “Canada 6” which is part of the Masters series. Create versions of
the chefdetails and judges tables that exclude any records coming from seasons that
include the word Masters or Canada.

In [4]:
chef_no_master_canada = df_select(
    "SELECT * FROM chefdetails where season not like '%Canada%' and season not like '%Master%';",
    engine)
chef_no_master_canada.sample(5)

Retrieved 322 records


,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
149,160,Michelle Minori,Michelle M.,None,San Francisco,California,NaN,Kentucky,16,US,4,POC,None,Female
116,127,Adrienne Cheatham,Adrienne C.,"Chicago, Illinois",Harlem,New York,NaN,Colorado,15,US,2,POC,Chef/Consultant,Female
290,325,Paul Qui,Paul Q.,None,Austin,Texas,31.0,Texas,9,US,1,POC,Executive Chef,Male
316,385,Dale MacKay,Dale MacKay,"Saskatoon, Saskatchewan",Saskatoon,Saskatchewan,NaN,World All Stars,20,US,10,None,Chef/Co-owner,Male
82,93,Stephanie Izard,Stephanie I.,"Stamford, Connecticut",Chicago,Illinois,31.0,Chicago,4,US,1,None,None,Female


In [5]:
judges_no_master_canada = df_select(
    "SELECT * FROM judges where season not like '%Canada%' and season not like '%Master%';",
    engine)
judges_no_master_canada.sample(5)

Retrieved 633 records


,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
19,19,All Stars: New York,8,US,10,Quickfire,Individual,Sesame St. characters,None,None
165,178,Chicago,4,US,5,Elimination,Team,Ming Tsai,None,Tournament of Champions Judge
8,8,All Stars: New York,8,US,4,Elimination,Team,Tony Mantuano,None,Top Chef Masters Season 2
565,663,Seattle,10,US,10,Quickfire,Individual,Wolfgang Puck,None,None
302,315,Kentucky,16,US,2,Quickfire,Individual,Gail Simmons,None,Top Chef: Just Desserts


2. Further trim your chefdetails and judges tables by including only the columns of
interest from the table descriptions above.

In [6]:
filtered_chef_trimmed = df_select(
    "SELECT season, seasonNumber, name, placement, gender FROM chefdetails where season not like '%Canada%' and season not like '%Master%';",
    engine)
filtered_chef_trimmed.sample(5)

Retrieved 322 records


,season,seasonNumber,name,placement,gender
103,D.C.,7,Amanda Baumgarten,6,Female
12,All Stars: New York,8,Casey Thompson,13,Female
241,New York,5,Richard Sweeney,14,Male
227,New Orleans,11,Ramon Bojorquez,19,Male
242,New York,5,Jill Snyder,15,Female


In [7]:
filtered_judges_trimmed = df_select(
    "SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC FROM judges where season not like '%Canada%' and season not like '%Master%';",
    engine)
filtered_judges_trimmed.sample(5)

Retrieved 633 records


,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
623,World All Stars,20,9,Elimination,Jimmy Famurewa,None
617,World All Stars,20,6,Elimination,Roger Pizey,None
19,All Stars: New York,8,10,Quickfire,Sesame St. characters,None
562,Seattle,10,9,Quickfire,Bob Kramer,None
72,Boston,12,1,Sudden Death Quickfire,Richard Blais,Yes


3. Your tables should have information from 20 remaining different seasons, numbered
from 1 to 20. Make a table that has two columns, one called season and one called
seasonNumber showing the number associated with each season. (For example, San
Francisco is season 1.)


In [8]:
with engine.connect() as conn:
    season_and_number = pd.read_sql(
        sqla.text("SELECT judges.season, judges.seasonNumber FROM judges join chefdetails on judges.season = chefdetails.season where judges.season not like '%Master%' and judges.season not like '%Canada%' order by seasonNumber;"), conn
    )
season_and_number

,season,seasonNumber
0,San Francisco,1
1,San Francisco,1
2,San Francisco,1
3,San Francisco,1
4,San Francisco,1
...,...,...
10174,World All Stars,20
10175,World All Stars,20
10176,World All Stars,20
10177,World All Stars,20


4. Answer the following questions using your tables so far.
a. Among all chef contestants, how many are male and how many are female?

In [9]:
filtered_chef_trimmed['gender'].value_counts()

Male      170
Female    152
Name: gender, dtype: int64

b. Among all winners (placement==1) how many are male and how many are female?

In [10]:
fct_win = filtered_chef_trimmed[filtered_chef_trimmed['placement'] == 1]
fct_win['gender'].value_counts()

Male      14
Female     6
Name: gender, dtype: int64

There are 6 female winners and 14 male winners.

c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many
are female?

In [11]:
fct_female = filtered_chef_trimmed[filtered_chef_trimmed['gender'] == 'Female']
fct_female_1 =fct_female.count('placement' == 1)
fct_female_2 =fct_female.count('placement' == 2)
fct_female_3 =fct_female.count('placement' == 3)
fct_female_top_3 = fct_female_1['gender']+ fct_female_2['gender'] +fct_female_3['gender']
fct_female_top_3

456

In [12]:
fct_male = filtered_chef_trimmed[filtered_chef_trimmed['gender'] == 'Male']
fct_male_1 =fct_male.count('placement' == 1)
fct_male_2 =fct_male.count('placement' == 2)
fct_male_3 =fct_male.count('placement' == 3)
fct_male_top_3 = fct_male_1['gender']+ fct_male_2['gender'] +fct_male_3['gender']
fct_male_top_3

510

There are 456 female values in the top 3 and 510 males in the top 3.

d. Compare the number of times a female was placed in the top 3 with the number of times
a female won. Does this number seem unusual? That is, are women who place in the
top 3 less likely than expected to ultimately win the competition? Why or why not?

In [13]:
fct_female_top_3 / (fct_female_top_3+fct_male_top_3) *100

47.20496894409938

In [14]:
6 / (6+14) * 100

30.0

In [15]:
6/fct_female_top_3 *100

1.3157894736842104

In [16]:
14/fct_male_top_3 * 100

2.7450980392156863

Out of everyone in the top 3, women make up 46.2% of the top 3. Out of those one, only 6 or 1.315% of have won TopChef. Since there are 6 women out of the 20 winners, this means that women have won 30% of the time. When we compare those number to the amount of time mean have won with the amount of men in the top 3, this puts women at as less likely to win competition. The amount of men in the top 3 is 54.8%, compared to the amount of women this isn't that big of a difference, however when you look at the amount of men in the top 3 that have won campared to the amount of women in the top 3 that have won, its 2.74% which is over double the amount of wins.

5. Make a table with one row for each first place winner containing the winner’s name,
the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they
served as a guest judge or not.

In [59]:
chef_guest_judge = filtered_judges_trimmed[filtered_judges_trimmed['competedOnTC'] == 'Yes']
chef_judge_list = []
for i in range(1, len(chef_guest_judge['guestJudge'])):
    if chef_guest_judge.iloc[i]['guestJudge'] not in chef_judge_list:
        chef_judge_list.append(chef_guest_judge.iloc[i]['guestJudge'])

In [60]:
chef_info = df_select("""SELECT season, seasonNumber, name
    FROM (select placement, name, season, seasonNumber from chefdetails where placement = 1) as subquery
    where season not like '%Canada%' and season not like '%Master%';""", engine)

Retrieved 20 records


In [61]:
chef_info['guest_judge?'] = chef_info['name'].apply(lambda x: 'Yes' if x in chef_judge_list else 'No')
chef_info

,season,seasonNumber,name,guest_judge?
0,All Stars: New York,8,Richard Blais,Yes
1,All-Stars L.A.,17,Melissa King,Yes
2,Boston,12,Mei Lin,No
3,California,13,Jeremy Ford,No
4,Charleston,14,Brooke Williamson,Yes
5,Chicago,4,Stephanie Izard,Yes
6,D.C.,7,Kevin Sbraga,No
7,Colorado,15,Joseph Flamm,No
8,Houston,19,Kah-wai Lo,No
9,Kentucky,16,Kelsey Barnard Clark,Yes


6. Find all contestants who were not first place winners but did serve as a guest judge at
some point. Make sure to only list each name once.


In [85]:
chef_not_win = df_select("""SELECT subquery.season, subquery.seasonNumber, subquery.name
    FROM (select distinct name, placement, season, seasonNumber from chefdetails where placement != 1) as subquery
    inner join judges on subquery.name = judges.guestJudge
    where subquery.season not like '%Canada%' and subquery.season not like '%Master%'group by subquery.name;""", engine)
chef_not_win

Retrieved 32 records


,season,seasonNumber,name
0,Seattle,10,Brooke Williamson
1,Chicago,4,Richard Blais
2,Chicago,4,Antonia Lofaso
3,New Orleans,11,Carlos Gaytan
4,World All Stars,20,Nicole Gomes
5,Los Angeles,2,Sam Talbot
6,Las Vegas,6,Bryan Voltaggio
7,Las Vegas,6,Mike Isabella
8,Chicago,4,Spike Mendelsohn
9,World All Stars,20,Dawn Burrell
